# Chapter 5 — Data Class Builders

## Overview of Data Class Builders

#### Example 5-1. [class/coordinates.py](class/coordinates.py)

In [1]:
class Coordinate:

    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

In [2]:
moscow = Coordinate(55.76, 37.62)
moscow

In [3]:
location = Coordinate(55.76, 37.62)
location == moscow

False

In [4]:
(location.lat, location.lon) == (moscow.lat, moscow.lon)

True

`Coordinate` class built with `collections.namedtuple`

In [5]:
from collections import namedtuple
Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [6]:
moscow = Coordinate(55.756, 37.617)
moscow

Coordinate(lat=55.756, lon=37.617)

In [7]:
moscow == Coordinate(55.756, 37.617)

True

`Coordinate` class built with `typing.NamedTuple`

In [8]:
import typing
Coordinate = typing.NamedTuple('Coordinate', [('lat', float), ('lon', float)])
# Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)
issubclass(Coordinate, tuple)

True

In [9]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

#### Example 5-2. [typing_namedtuple/coordinates.py](typing_namedtuple/coordinates.py)

In [10]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

In [11]:
try:
    issubclass(Coordinate, typing.NamedTuple)
except TypeError as err:
    print(err)

issubclass() arg 2 must be a class, a tuple of classes, or a union


In [12]:
issubclass(Coordinate, tuple)

True

#### Example 5-3. [dataclass/coordinates.py](dataclass/coordinates.py)

In [13]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

In [14]:
issubclass(Coordinate, object)

True

## Classic Named Tuples

#### Example 5-4. Defining and using a named tuple type

In [15]:
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [16]:
tokyo.population

36.933

In [17]:
tokyo.coordinates

(35.689722, 139.691667)

In [18]:
tokyo[1]

'JP'

#### Example 5-5. Named tuple attributes and methods (continued from the previous example)

In [19]:
City._fields

('name', 'country', 'population', 'coordinates')

In [20]:
Coordinate = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

In [21]:
import json
json.dumps(delhi._asdict())

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

#### Example 5-6. Named tuple attributes and methods, continued from Example 5-5.

In [22]:
Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=['WGS84'])
Coordinate(0, 0)

Coordinate(lat=0, lon=0, reference='WGS84')

In [23]:
Coordinate._field_defaults

{'reference': 'WGS84'}

### Hacking a `namedtuple` to inject a method

#### Example 5-7. Adding a class attribute and a method to `Card`, the `namedtuple` from "A Pythonic Card Deck"

In [24]:
import collections


Card = collections.namedtuple('Card', ['rank', 'suit'])
Card.suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    suit_value = card.suit_values[card.suit]
    return rank_value * len(card.suit_values) + suit_value


Card.overall_rank = spades_high
lowest_card = Card('2', 'clubs')
highest_card = Card('A', 'spades')

In [25]:
lowest_card.overall_rank()

0

In [26]:
highest_card.overall_rank()

51

## Typed Named Tuples

#### Example 5-8. [typing_namedtuple/coordinates2.py](typing_namedtuple/coordinates2.py)

In [27]:
class Coordinate(NamedTuple):
    lat: float
    lon: float
    reference: str = 'WGS84'

## Type Hints 101

#### Example 5-9. Python does not enforce type hints at runtime

In [28]:
class Coordinate(typing.NamedTuple):
    lat: float
    lon: float

In [29]:
trash = Coordinate('Ni', None)
print(trash)

Coordinate(lat='Ni', lon=None)


### The Meaning of Variable Annotations

#### Example 5-10. [meaning/demo_plain.py](meaning/demo_plain.py): a plain class with type hints

In [30]:
class DemoPlainClass:
    a: int
    b: float = 1.1
    c = 'spam'

In [31]:
DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [32]:
DemonPlainClass.a

NameError: name 'DemonPlainClass' is not defined

In [33]:
DemoPlainClass.b

1.1

In [34]:
DemoPlainClass.c

'spam'

#### Example 5-11. [meaning/demo_nt.py](meaning/demo_nt.py): a class built with `typing.NamedTuple`

In [35]:
class DemoNTClass(typing.NamedTuple):
    a: int
    b: float = 1.1
    c = 'spam'

In [36]:
DemoNTClass.__annotations__

{'a': int, 'b': float}

In [37]:
DemoNTClass.a

_tuplegetter(0, 'Alias for field number 0')

In [38]:
DemoNTClass.b

_tuplegetter(1, 'Alias for field number 1')

In [39]:
DemoNTClass.c

'spam'

In [40]:
DemoNTClass.__doc__

'DemoNTClass(a, b)'

In [41]:
nt = DemoNTClass(8)
nt.a

8

In [42]:
nt.b

1.1

In [43]:
nt.c

'spam'

In [44]:
nt.a = 1

AttributeError: can't set attribute

In [45]:
nt.b = 17.3

AttributeError: can't set attribute

In [46]:
nt.c = 'spam2'

AttributeError: 'DemoNTClass' object attribute 'c' is read-only

#### Example 5-12. [meaning/demo_dc.py](meaning/demo_dc.py): a class decorated with `@dataclass`

In [47]:
from dataclasses import dataclass

@dataclass
class DemoDataClass:
    a: int
    b: float = 1.1
    c = 'spam'

In [48]:
DemoDataClass.__annotations__

{'a': int, 'b': float}

In [49]:
DemoDataClass.__doc__

'DemoDataClass(a: int, b: float = 1.1)'

In [50]:
DemoDataClass.a

AttributeError: type object 'DemoDataClass' has no attribute 'a'

In [51]:
DemoDataClass.b

1.1

In [52]:
DemoDataClass.c

'spam'

In [53]:
dc = DemoDataClass(9)
dc.a

9

In [54]:
dc.b

1.1

In [55]:
dc.c

'spam'

No type checking is done at runtime

In [56]:
dc.a = 10
dc.b = 'oops'
dc.c = 'whatever'
dc.z = 'secret stash'

## More about `@dataclass`

### Field Options

#### Example 5-13. [dataclass/club_wrong.py](dataclass/club_wrong.py): this class raises `ValueError`

In [57]:
try:
    @dataclass
    class ClubMember:
        name: str
        guests: list = []
except ValueError as err:
    print(f"ValueError: {err}")

ValueError: mutable default <class 'list'> for field guests is not allowed: use default_factory


#### Example 5-14. [dataclass/club.py](dataclass/club.py): this `ClubMember` definition works

In [58]:
from dataclasses import field


@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)

#### Example 5-15. [dataclass/club_generic.py](dataclass/club_generic.py): this `ClubMember` definition is more precise

In [59]:
@dataclass
class ClubMember:
    name: str
    guests: list[str] = field(default_factory=list)
    athelete: bool = field(default=False, repr=False)  # Omitted from __repr__

### Post-init Processing

#### Example 5-16. [dataclass/hackerclub.py](dataclass/hackerclub.py)

In [60]:
from dataclass.club import ClubMember


@dataclass
class HackerClubMember(ClubMember):
    all_handles = set()
    # all_handles: ClassVar[set[str]] = set()  # For mypy type checking
    handle: str = ''
        
    def __post_init__(self):
        cls = self.__class__
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:
            msg = f'handle {self.handle!r} already exists.'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)
        
HackerClubMember.__doc__

"HackerClubMember(name: str, guests: list = <factory>, handle: str = '')"

In [61]:
anna = HackerClubMember('Anna Ravenscroft', handle='AnnaRaven')
anna

HackerClubMember(name='Anna Ravenscroft', guests=[], handle='AnnaRaven')

In [62]:
leo = HackerClubMember('Leo Rochael')
leo

HackerClubMember(name='Leo Rochael', guests=[], handle='Leo')

In [63]:
try:
    leo2 = HackerClubMember('Leo DaVinci')
except ValueError as err:
    print(f"ValueError: {err}")

ValueError: handle 'Leo' already exists.


### Initialization Variables That Are Not Fields

#### Example 5-18. Example from [`dataclasses`](https://docs.python.org/3/library/dataclasses.html#init-only-variables) module documentation
https://docs.python.org/3/library/dataclasses.html#init-only-variables

### `@dataclass` Example: Dublin Core Resource Record

#### Example 5-19. [dataclass/resource.py](dataclass/resource.py): code for `Resource`, a class based on Dublin Core terms

In [64]:
from dataclasses import dataclass, field, fields
from typing import Optional
from enum import Enum, auto
from datetime import date


class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()
    

@dataclass
class Resource:
    """Media resource description."""
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)
        
    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']
        for f in fields(cls):
            value = getattr(self, f.name)
            res.append(f'{indent}{f.name} = {value!r},')
        
        res.append(')')
        return '\n'.join(res)    

#### Example 5-20. [dataclass/resource.py](dataclass/resource.py): code for `Resource`, a class based on Dublin Core terms

In [65]:
description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd edition',
    ['Martin Fowler', 'Kent Back'], date(2018, 11, 19),
    ResourceType.BOOK, description, 'EN',
    ['computer programming', 'OOP'])
book

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd edition',
    creators = ['Martin Fowler', 'Kent Back'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

## Pattern Matching Class Instances

### Keyword Class Patterns

#### Example 5-22. `City` class and a few instances

In [66]:
import typing


class City(typing.NamedTuple):
    continent: str
    name: str
    country: str
        
cities = [
    City('Asia', 'Toyko', 'JP'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('South America', 'São Paulo', 'BR'),    
]

In [67]:
def match_asian_cities():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia'):
                results.append(city)
    return results


match_asian_cities()

[City(continent='Asia', name='Toyko', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [68]:
def match_asian_countries():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia', country=cc):
                results.append(cc)
    return results


match_asian_countries()

['JP', 'IN']

In [69]:
def match_asian_cities_pos():
    results = []
    for city in cities:
        match city:
            case City('Asia', _, country):
                results.append(country)
    return results

match_asian_cities_pos()

['JP', 'IN']

In [70]:
City.__match_args__

('continent', 'name', 'country')